# Speed Skydiving scoring bulk files processor
Uses the `./data` directory as a data lake.  It finds all valid FlySight files in that directory, from 0 to as many as it finds and fit in memory, and process them all.  Summary of "all files" or "all jumps" at the bottom of the document.

In [1]:
!make local

pip install -e .
Obtaining file:///home/jovyan
  Installing build dependencies ... done
  Checking if build backend supports build_editable ... done
  Getting requirements to build editable ... done
  Preparing editable metadata (pyproject.toml) ... done
  Using cached haversine-2.8.1-py2.py3-none-any.whl.metadata (5.9 kB)
Using cached haversine-2.8.1-py2.py3-none-any.whl (7.7 kB)
  Building editable for ssscoring (pyproject.toml) ... done
  Created wheel for ssscoring: filename=ssscoring-1.7.0-0.editable-py3-none-any.whl size=4383 sha256=a9c225c9daa3e7b78fbb7859ef87fa28643586624dce7d31dc1127376dde82e3
  Stored in directory: /tmp/pip-ephem-wheel-cache-yjkmf1hz/wheels/d5/0b/b6/3714fd2d5297ec16b9fe56b0d4ce5d9925529ac2d0c67d4765
Successfully built ssscoring


In [2]:
from collections import namedtuple
from copy import deepcopy

from ssscoring.calc import aggregateResults
from ssscoring.calc import convertFlySight2SSScoring
from ssscoring.calc import dropNonSkydiveDataFrom
from ssscoring.calc import getSpeedSkydiveFrom
from ssscoring.calc import isValidJump
from ssscoring.calc import isValidMinimumAltitude
from ssscoring.calc import jumpAnalysisTable
from ssscoring.calc import processAllJumpFiles
from ssscoring.calc import roundedAggregateResults
from ssscoring.calc import totalResultsFrom
from ssscoring.constants import BREAKOFF_ALTITUDE
from ssscoring.constants import FT_IN_M
from ssscoring.constants import PERFORMANCE_WINDOW_LENGTH
from ssscoring.flysight import getAllSpeedJumpFilesFrom
from ssscoring.flysight import validFlySightHeaderIn
from ssscoring.notebook import SPEED_COLORS
from ssscoring.notebook import graphAltitude
from ssscoring.notebook import graphAngle
from ssscoring.notebook import graphJumpResult
from ssscoring.notebook import initializeExtraYRanges
from ssscoring.notebook import initializePlot

import csv
import os
import os.path as path

import bokeh.plotting as bp
import bokeh.models as bm
import ipywidgets as widgets
import pandas as pd

In [3]:
DATA_LAKE_ROOT = './data'

---
## <span style="color: yellow">IMPORTANT</span> - Set the drop zone altitude MSL

Set the value in ft (even though SSScoring uses meters) out of convenience because all DZs in the US, many in Asia and Mexico, and many in Europe express the altitude in feet.

### Some DZs

| Drop zone | Alt (ft) |
|-----------|----------|
| Bay Area Skydiving | 23 |
| SkyDance SkyDiving | 100 |
| Skydive Arizona | 1509 |
| Skydive Chicago | 616 |
| Thai Sky Adventures | 21 |

In [4]:
dropZoneAltMSL = 23

In [5]:
dropZoneAltMSLMeters = dropZoneAltMSL/FT_IN_M
display(widgets.HTML('<h2>DZ Altitude = <span style = "color: green">%7.2f ft</span> (%7.2f m)<h1>' % (dropZoneAltMSL, dropZoneAltMSLMeters)))

HTML(value='<h2>DZ Altitude = <span style = "color: green">  23.00 ft</span> (   7.01 m)<h1>')

In [6]:
jumpFiles = getAllSpeedJumpFilesFrom(DATA_LAKE_ROOT)

In [7]:
jumpResults = processAllJumpFiles(jumpFiles, altitudeDZMeters = dropZoneAltMSLMeters)

/opt/conda/lib/python3.11/site-packages/numba/cpython/hashing.py:482: UserWarning: FNV hashing is not implemented in Numba. See PEP 456 https://www.python.org/dev/peps/pep-0456/ for rationale over not using FNV. Numba will continue to work, but hashes for built in types will be computed using siphash24. This will permit e.g. dictionaries to continue to behave as expected, however anything relying on the value of the hash opposed to hash as a derived property is likely to not work as expected.
  warnings.warn(msg)


### Air density

**<a href='https://aviex.goflexair.com/blog/density-of-air-air-density' target='_blank'>Pilot's Guide to Air Density</a>** - A useful guide with calculations, lots of information.

![](https://ciurana.eu/personal/air-density-partial-table.png)

---
## Results

In [8]:
aggregate = aggregateResults(jumpResults)
sumResults = totalResultsFrom(aggregate)
display(aggregate)

,score,5.0,10.0,15.0,20.0,25.0,finalTime,maxSpeed
00-00-00-baseline-20240928:v1,488.60496,177.3720,316.4400,415.2240,472.3200,492.8760,24.5,492.8760
09-29-08:v2,483.437845,172.6812,312.7716,414.6012,472.5576,481.8456,24.8,485.0388
10-50-38:v2,489.617187,177.5628,315.5868,416.2572,475.4628,494.0820,24.6,494.0820
12-22-54:v2,487.584929,181.8900,328.6008,416.8728,475.8624,487.2384,24.5,489.4992


### Rounded results for training log

In [9]:
roundedResults = roundedAggregateResults(aggregate)

### All jumps

In [10]:
def dumpSimple(table: pd.DataFrame):
    fileName = os.path.join(DATA_LAKE_ROOT, resultRef+'.tsv')
    simpleTable = table.copy()
    simpleTable = simpleTable.drop('speedAngle', axis = 1)
    simpleTable.to_csv(fileName, sep = '\t', index = False)

In [11]:
def displayJumpDataIn(resultsTable: pd.DataFrame):
    table = resultsTable.copy()
    # Experimental
    # For more information on the `interpolate` method and its options, see the [pandas documentation](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.interpolate.html).
    # Additionally, you can also consider using other libraries like `scipy` which provides more advanced interpolation methods, such as `interp1d` or `griddata`. See the [scipy documentation](https://docs.scipy.org/doc/scipy/reference/interpolate.html) for more information.
    table.vKMh = table.vKMh.apply(round)
    table.hKMh = table.hKMh.apply(round)
    table['altitude (ft)'] = table['altitude (ft)'].apply(round)
    table.netVectorKMh = table.netVectorKMh.apply(round)
    table.index = ['']*len(table)
    display(table)
    # dumpSimple(table)

In [12]:
allJumpsPlot = initializePlot('All jumps in set')
jumpNumber = 0
mixColor = 0
for resultRef in sorted(jumpResults.keys()):
    jumpResult = jumpResults[resultRef]
    if jumpResult.score > 0.0:
        validJumpStatus = '<hr><h1><span style="color: %s">%s jump - %s - score = %.02f km/h</span></h1>' % (jumpResult.color, resultRef, jumpResult.result, jumpResult.score)        
    else:
        validJumpStatus = '<hr><h1><span style="color: %s">%s jump - %s</span></h1>' % (jumpResult.color, resultRef, jumpResult.result)
    display(widgets.HTML(validJumpStatus))

    maxSpeed = jumpResult.maxSpeed
    window = jumpResult.window
    mixColor = (mixColor+1)%len(SPEED_COLORS)
    if jumpResult.score > 0.0:
        display(widgets.HTML('<h3>Max speed = {0:,.0f}; '.format(maxSpeed)+('exit at %d m (%d ft), end scoring window at %d m (%d ft)'%(window.start, 3.2808*window.start, window.end, 3.2808*window.end))))
        if not isValidMinimumAltitude(jumpResult.data.altitudeAGL.max()):
            display(widgets.HTML('<span style="color: yellow"><span style="font-weight: bold">Warning:</span> exit altitude ASL was lower than the minimum scoring altitude according to IPC and USPA.'))
        display(widgets.HTML('</h3>'))
        # TODO: Move this to an in-notebook function or module.
        displayJumpDataIn(jumpResult.table)
        individualPlot = initializePlot(resultRef)
        individualPlot = initializeExtraYRanges(individualPlot, startY = min(jumpResult.data.altitudeAGLFt)-500.0, endY = max(jumpResult.data.altitudeAGLFt)+500.0)
        graphAltitude(individualPlot, jumpResult)
        graphAngle(individualPlot, jumpResult)
        hoverValue = bm.HoverTool(tooltips = [ ('Y-val', '@y{0.00}'), ])
        individualPlot.add_tools(hoverValue)
        graphJumpResult(individualPlot,
                        jumpResult,
                        lineColor = SPEED_COLORS[0])
        graphJumpResult(allJumpsPlot,
                        jumpResult,
                        lineColor = SPEED_COLORS[mixColor],
                        legend = '%s - %.2f' % (resultRef, jumpResult.score),
                        showIt = False)

HTML(value='<hr><h1><span style="color: #0f0">00-00-00-baseline-20240928:v1 jump - 🟢 valid - score = 488.60 km…

HTML(value='<h3>Max speed = 493; exit at 4344 m (14252 ft), end scoring window at 2088 m (6851 ft)')

HTML(value='</h3>')

,time,vKMh,hKMh,speedAngle,distanceFromExit,altitude (ft),netVectorKMh
,5.0,177,183,44.1,290.800000,13821,255
,10.0,316,163,62.7,526.500000,12666,356
,15.0,415,121,73.8,725.200000,10975,432
,20.0,472,84,80.0,867.800000,8923,480
,25.0,493,65,82.5,964.878149,6928,497


HTML(value='<hr><h1><span style="color: #0f0">09-29-08:v2 jump - 🟢 valid - score = 483.44 km/h</span></h1>')

...

,time,vKMh,hKMh,speedAngle,distanceFromExit,altitude (ft),netVectorKMh
,5.0,178,183,44.2,290.800000,13829,255
,10.0,316,163,62.7,527.200000,12671,355
,15.0,416,121,73.8,726.100000,10980,433
,20.0,475,88,79.5,870.100000,8923,484
,25.0,494,63,82.7,968.919436,6882,498


HTML(value='<hr><h1><span style="color: #0f0">12-22-54:v2 jump - 🟢 valid - score = 487.58 km/h</span></h1>')

HTML(value='<h3>Max speed = 489; exit at 4186 m (13734 ft), end scoring window at 1930 m (6333 ft)')

HTML(value='</h3>')

,time,vKMh,hKMh,speedAngle,distanceFromExit,altitude (ft),netVectorKMh
,5.0,182,178,45.6,280.500000,13298,255
,10.0,329,143,66.4,502.200000,12108,359
,15.0,417,124,73.4,690.100000,10387,435
,20.0,476,101,78.0,848.700000,8332,486
,25.0,487,67,82.1,954.171777,6339,492


---
## Summary of all speed skydives in the data lake

**<a href='https://intimescoring.com/Results/EventResults?dbid=100&mid=1074&smid=0&mdcid=1273&dseid=0' target='_blank'>InTime Scoring - 2024 USPA Speed Open results</a>**

In [13]:
display(roundedResults)
display(sumResults)
bp.show(allJumpsPlot)

,score,5.0,10.0,15.0,20.0,25.0,finalTime,maxSpeed
00-00-00-baseline-20240928:v1,489,177,316,415,472,493,24.5,493
09-29-08:v2,483,173,313,415,473,482,24.8,485
10-50-38:v2,490,178,316,416,475,494,24.6,494
12-22-54:v2,488,182,329,417,476,487,24.5,489


,totalSpeed,meanSpeed,maxScore
totalSpeed,1949.244921,487.31123,489.617187
